<a href="https://colab.research.google.com/github/Nguyenanh393/Optima/blob/main/milp_bsc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pulp
from pulp import *

In [ ]:
# Đặt tên bài toán
problem = LpProblem("BloodSupplyChain_CO2", LpMinimize)

In [ ]:
# Đặt tên các biến

# Số ngày
num_periods = 29

# Nhóm máu
blood_types = ['A', 'B', 'AB', 'O']

# Ngân hàng máu
regional_blood_banks = "VienHuyetHocTrungUong"

# Điểm cung cấp máu lưu động
MUFs = ["UBNDPhuongNguyenDu", "TieuHocGiaQuat"] # dữ liệu ngày (26/11/2023)

# Điểm cung cấp máu cố định
FUFs = ["26LuongNgocQuyen", "132QuanNhan", "10/122/DuongLang", "78NguyenTruongTo", "KM13+500QL1ANgocHoi"]

# Điểm tiêu thụ máu
hospitals = ["BVBachMai", "BVPhuSanTrungUong","BVTimHaNoi"]



In [ ]:
# Parameters

# Khoảng cách từ MUFs đến RBBS
X_ir = {("UBNDPhuongNguyenDu", "VienHuyetHocTrungUong"): 9,
        ("TieuHocGiaQuat", "VienHuyetHocTrungUong"): 25}

# Khoảng cách từ FUFs đến RBBS
X_jr = {("26LuongNgocQuyen", "VienHuyetHocTrungUong"): 10,
        ("132QuanNhan", "VienHuyetHocTrungUong"): 5,
        ("10/122/DuongLang", "VienHuyetHocTrungUong"): 5,
        ("78NguyenTruongTo", "VienHuyetHocTrungUong"): 9,
        ("KM13+500QL1ANgocHoi", "VienHuyetHocTrungUong"): 35}

# Khoảng cách từ MUFs đến Hs
X_ih = {("UBNDPhuongNguyenDu", "BVBachMai"): 5, ("UBNDPhuongNguyenDu", "BVPhuSanTrungUong"): 5, ("UBNDPhuongNguyenDu", "BVTimHaNoi"): 5,
        ("TieuHocGiaQuat", "BVBachMai"): 5, ("TieuHocGiaQuat", "BVPhuSanTrungUong"): 5, ("TieuHocGiaQuat", "BVTimHaNoi"): 5}

# Khoảng cách từ FUFs đến Hs
X_jh = {("26LuongNgocQuyen", "BVBachMai"): 7, ("26LuongNgocQuyen", "BVPhuSanTrungUong"): 2, ("26LuongNgocQuyen", "BVTimHaNoi"): 4,
        ("132QuanNhan", "BVBachMai"): 5, ("132QuanNhan", "BVPhuSanTrungUong"): 6, ("132QuanNhan", "BVTimHaNoi"): 7,
        ("10/122/DuongLang", "BVBachMai"): 4, ("10/122/DuongLang", "BVPhuSanTrungUong"): 6, ("10/122/DuongLang", "BVTimHaNoi"): 5,
        ("78NguyenTruongTo", "BVBachMai"): 6, ("78NguyenTruongTo", "BVPhuSanTrungUong"): 3, ("78NguyenTruongTo", "BVTimHaNoi"): 3,
        ("KM13+500QL1ANgocHoi", "BVBachMai"): 11, ("KM13+500QL1ANgocHoi", "BVPhuSanTrungUong"): 14, ("KM13+500QL1ANgocHoi", "BVTimHaNoi"): 12}

# Khoảng cách từ RBBS đến Hs
X_rh = {("VienHuyetHocTrungUong", "BVBachMai"): 8, ("VienHuyetHocTrungUong", "BVPhuSanTrungUong"): 8, ("VienHuyetHocTrungUong", "BVTimHaNoi"): 8}

# Mức chứa tối đa nhóm máu p tại RBBs (túi)
Z_pr = 10000

# Mức chứa tối đa nhóm máu p tại Hs (túi)
Z_ph = 2000

CAP = 10000
alpha = 25

# Nhu cầu về nhóm máu p tại các bệnh viện h
D_ph = {('A', "BVBachMai"): 1000, ('B', "BVBachMai"): 800,
        ('AB', "BVBachMai"): 900, ('O', "BVBachMai"): 700,
        ('A', "BVPhuSanTrungUong"): 500, ('B', "BVPhuSanTrungUong"): 1100,
        ('AB', "BVPhuSanTrungUong"): 900, ('O', "BVPhuSanTrungUong"): 800,
        ('A', "BVTimHaNoi"): 900, ('B', "BVTimHaNoi"): 800,
        ('AB', "BVTimHaNoi"): 600, ('O', "BVTimHaNoi"): 700}

In [ ]:
# import random as rd
# print(round(rd.uniform(0.002, 0.008),3))
# print(round(rd.uniform(0.002, 0.008),3))
# print(round(rd.uniform(0.00002, 0.00008),5))
# print(round(rd.uniform(0.00002, 0.00008),5))
# print(round(rd.uniform(0.00002, 0.00008),5))
# print(round(rd.uniform(0.00002, 0.00008),5))
# print(round(rd.uniform(0.00002, 0.00008),5))

EP_pr = {'A' : 0.004, 'B' : 0.006, 'AB' : 0.005, 'O' : 0.003}
EP_ph = {'A' : 0.004, 'B' : 0.007, 'AB' : 0.003, 'O' : 0.002}

ER_pir = {'A' : 0.00003, 'B' : 0.00005, 'AB' : 0.00004, 'O' : 0.00006}
ER_pjr = {'A' : 0.00006, 'B' : 0.00003, 'AB' : 0.00004, 'O' : 0.00004}
ER_pih = {'A' : 0.00007, 'B' : 0.00004, 'AB' : 0.00008, 'O' : 0.00004}
ER_pjh = {'A' : 0.00006, 'B' : 0.00007, 'AB' : 0.00002, 'O' : 0.00005}
ER_prh = {'A' : 0.00004, 'B' : 0.00007, 'AB' : 0.00004, 'O' : 0.00003}


In [ ]:
# Decision Variables

# Số lượng máu nhóm p được sản xuất tại r trong thời gian t (túi) (Giả sử >= 100)
G_pr = LpVariable.dicts("G_pr", (range(num_periods), blood_types, regional_blood_banks), 0, None, LpInteger)

# Số lượng máu nhóm p được sản xuất tại h trong thời gian t (túi) (>= 40)
G_ph = LpVariable.dicts("G_ph", (range(num_periods), blood_types, hospitals), 0, None, LpInteger)

# Số lượng máu p được vận chuyển từ MUFs -> RBBs trong thời gian t (túi) (>= 10)
Q_pir = LpVariable.dicts("Q_pir", (range(num_periods), blood_types, MUFs, regional_blood_banks), 0, None, LpInteger)

# Số lượng máu p được vận chuyển từ FUFs -> RBBs trong thời gian t (túi) (>= 10)
Q_pjr = LpVariable.dicts("Q_pjr", (range(num_periods), blood_types, FUFs, regional_blood_banks), 0, None, LpInteger)

# Số lượng máu p được vận chuyển từ MUFs -> Hs trong thời gian t (túi) (>= 10)
Q_pih = LpVariable.dicts("Q_pih", (range(num_periods), blood_types, MUFs, hospitals), 0, None, LpInteger)

# Số lượng máu p được vận chuyển từ FUFs -> Hs trong thời gian t (túi) (>= 10)
Q_pjh = LpVariable.dicts("Q_pjh", (range(num_periods), blood_types, FUFs, hospitals), 0, None, LpInteger)

# Số lượng máu p được vận chuyển từ RBBs -> Hs trong thời gian t (túi) (>= 25)
Q_prh = LpVariable.dicts("Q_prh", (range(num_periods), blood_types, regional_blood_banks, hospitals), 0, None, LpInteger)

# Số lượng máu tồn kho tại RBBs trong thời gian t (túi)
I_pr = LpVariable.dicts("I_pr", (range(num_periods), blood_types, regional_blood_banks), 0, None, LpInteger)

# Số lượng máu tồn kho tại Hs trong thời gian t (túi)
I_ph = LpVariable.dicts("I_ph", (range(num_periods), blood_types, hospitals), 0, None, LpInteger)

# Số lượng máu hết hạn tại RBBs trong thời gian t (túi)
W_pr = LpVariable.dicts("W_pr", (range(num_periods), blood_types, regional_blood_banks), 0, None, LpInteger)

# Số lượng máu hết hạn tại Hs trong thời gian t (túi)
W_ph = LpVariable.dicts("W_ph", (range(num_periods), blood_types, hospitals), 0, None, LpInteger)

# Nếu MUFS được sử dụng trong thời gian t thì = 1, ngược lại = 0
y_i = LpVariable.dicts("y_i", (range(num_periods), MUFs), 0, 1, LpInteger)

# Nếu FUFs được sử dụng trong thời gian t thì = 1, ngược lại = 0
y_j = LpVariable.dicts("y_j", (range(num_periods), FUFs), 0, 1, LpInteger)

# Nếu RBBs được sử dụng trong thời gian t thì = 1, ngược lại = 0
y_r = LpVariable.dicts("y_r", (range(num_periods), regional_blood_banks), 0, 1, LpInteger)


In [ ]:
# hàm mục tiêu : min carbon
problem += lpSum([lpSum([lpSum([EP_pr[p] * G_pr[t][p][r] for p in blood_types]) for r in regional_blood_banks]) for t in range(num_periods)]) + \
            lpSum([lpSum([lpSum([EP_ph[p] * G_ph[t][p][h] for p in blood_types]) for h in hospitals]) for t in range(num_periods)]) + \
            lpSum([lpSum([lpSum([lpSum([ER_pir[p] * Q_pir[t][p][i][r] for p in blood_types]) for i in MUFs]) for r in regional_blood_banks]) for t in range(num_periods)]) + \
            lpSum([lpSum([lpSum([lpSum([ER_pjr[p] * Q_pjr[t][p][j][r] for p in blood_types]) for j in FUFs]) for r in regional_blood_banks]) for t in range(num_periods)]) + \
            lpSum([lpSum([lpSum([lpSum([ER_pih[p] * Q_pih[t][p][i][h] for p in blood_types]) for i in MUFs]) for h in hospitals]) for t in range(num_periods)]) + \
            lpSum([lpSum([lpSum([lpSum([ER_pjh[p] * Q_pjh[t][p][j][h] for p in blood_types]) for j in FUFs]) for h in hospitals]) for t in range(num_periods)])+ \
            lpSum([lpSum([lpSum([lpSum([ER_prh[p] * Q_prh[t][p][r][h] for p in blood_types]) for r in regional_blood_banks]) for h in hospitals]) for t in range(num_periods)]), "Total_Carbon_Footprint"

In [ ]:
# Ràng buộc

# Số dư dự trữ của các nhóm máu tại các ngân hàng máu khu vực hiện có r, bệnh viện h trong thời gian t
for t in range(1, num_periods):
    for p in blood_types:
        for r in regional_blood_banks:
            problem += I_pr[t][p][r] == I_pr[t-1][p][r] + lpSum([Q_pir[t][p][i][r] for i in MUFs]) + \
                                                          lpSum([Q_pjr[t][p][j][r] for j in FUFs]) - \
                                                          lpSum([Q_prh[t][p][r][h] for h in hospitals])+ W_pr[t][p][r]

for t in range(1, num_periods):
    for p in blood_types:
        for h in hospitals:
            problem += I_ph[t][p][h] == I_ph[t-1][p][h] + lpSum([Q_pih[t][p][i][h] for i in MUFs]) + \
                                                          lpSum([Q_pjh[t][p][j][h] for j in FUFs]) + \
                                                          lpSum([Q_prh[t][p][r][h] for r in regional_blood_banks]) + W_ph[t][p][h] - \
                                                          D_ph[(p, h)]

for i in MUFs:
    problem += lpSum(y_i[t][i] for t in range(num_periods)) >= 1

for j in FUFs:
    problem += lpSum(y_j[t][j] for t in range(num_periods)) >= 1

for r in regional_blood_banks:
    problem += lpSum(y_r[t][r] for t in range(num_periods)) >= 1

for t in range(num_periods):
    for p in blood_types:
        for r in regional_blood_banks:
            problem += lpSum([Q_pir[t][p][i][r] for i in MUFs]) + lpSum([Q_pjr[t][p][j][r] for j in FUFs]) == G_pr[t][p][r]

for t in range(num_periods):
    for p in blood_types:
        for h in hospitals:
            problem += lpSum([Q_pih[t][p][i][h] for i in MUFs]) + lpSum([Q_pjh[t][p][j][h] for j in FUFs]) + lpSum([Q_prh[t][p][r][h] for r in regional_blood_banks]) == G_ph[t][p][h]

for t in range(num_periods):
    for p in blood_types:
        for h in hospitals:
            problem += lpSum([Q_pih[t][p][i][h] for i in MUFs]) + \
                        lpSum([Q_pjh[t][p][j][h] for j in FUFs]) + \
                        lpSum([Q_prh[t][p][r][h] for r in regional_blood_banks]) <= D_ph[(p,h)]

for t in range(num_periods):
    for p in blood_types:
        for r in regional_blood_banks:
            problem += I_pr[t][p][r] <= Z_pr


for t in range(num_periods):
    for p in blood_types:
        for h in hospitals:
            problem += I_ph[t][p][h] <= Z_ph




for t in range(alpha + 1, num_periods):
    for p in blood_types:
        for r in regional_blood_banks:
            temp_expr = I_pr[t - alpha][p][r] - lpSum([Q_pir[t - alpha][p][i][r] for i in MUFs]) - lpSum([Q_pjr[t - alpha][p][j][r] for j in FUFs]) - W_pr[t - alpha][p][r]
            problem += W_pr[t][p][r] >= 0
            problem += W_pr[t][p][r] >= temp_expr

for t in range(alpha, num_periods):
    for p in blood_types:
        for h in hospitals:
            temp_expr = I_ph[t - alpha][p][h] - lpSum([Q_pih[t - alpha][p][i][h] for i in MUFs]) - lpSum([Q_pjh[t - alpha][p][j][h] for j in FUFs]) - lpSum([Q_prh[t - alpha][p][r][h] for r in regional_blood_banks]) - W_ph[t - alpha][p][h]
            problem += W_ph[t][p][h] >= 0
            problem += W_ph[t][p][h] >= temp_expr

problem += lpSum([lpSum([lpSum(EP_pr[p] * G_pr[t][p][r] for p in blood_types) for r in regional_blood_banks]) for t in range(num_periods)]) + \
            lpSum([lpSum([lpSum(EP_ph[p] * G_ph[t][p][h] for p in blood_types) for h in hospitals]) for t in range(num_periods)]) + \
            lpSum([lpSum([lpSum([ER_pir[p] * Q_pir[t][p][i][r] for p in blood_types]) for i in MUFs]) for r in regional_blood_banks]) + \
            lpSum([lpSum([lpSum([ER_pjr[p] * Q_pjr[t][p][j][r] for p in blood_types]) for j in FUFs]) for r in regional_blood_banks]) + \
            lpSum([lpSum([lpSum([ER_pih[p] * Q_pih[t][p][i][h] for p in blood_types]) for i in MUFs]) for h in hospitals]) + \
            lpSum([lpSum([lpSum([ER_pjh[p] * Q_pjh[t][p][j][h] for p in blood_types]) for j in FUFs]) for h in hospitals]) + \
            lpSum([lpSum([lpSum([ER_prh[p] * Q_prh[t][p][r][h] for p in blood_types]) for r in regional_blood_banks]) for h in hospitals]) <= CAP * 4

for t in range(num_periods):
    for p in blood_types:
        for r in regional_blood_banks:
            problem += G_pr[t][p][r] >= 100

for t in range(num_periods):
    for p in blood_types:
        for h in hospitals:
            problem += G_ph[t][p][h] >= 40

for t in range(num_periods):
    for p in blood_types:
        for i in MUFs:
            problem += Q_pir[t][p][i][r] >= 10

for t in range(num_periods):
    for p in blood_types:
        for j in FUFs:
            problem += Q_pjr[t][p][j][r] >= 10

for t in range(num_periods):
    for p in blood_types:
        for i in MUFs:
            problem += Q_pih[t][p][i][h] >= 10

for t in range(num_periods):
    for p in blood_types:
        for j in FUFs:
            problem += Q_pjh[t][p][j][h] >= 10

for t in range(num_periods):
    for p in blood_types:
        for r in regional_blood_banks:
            problem += Q_prh[t][p][r][h] >= 25

for t in range(num_periods):
    for p in blood_types:
        for r in regional_blood_banks:
            problem += I_pr[t][p][r] >= 0

for t in range(num_periods):
    for p in blood_types:
        for h in hospitals:
            problem += I_ph[t][p][h] >= 0

for t in range(num_periods):
    for p in blood_types:
        for r in regional_blood_banks:
            problem += W_pr[t][p][r] >= 0

for t in range(num_periods):
    for p in blood_types:
        for h in hospitals:
            problem += W_ph[t][p][h] >= 0







In [ ]:
# Giải bài toán
problem.solve()

# In kết quả
print("Status:", LpStatus[problem.status])
print("Total Carbon Footprint = ", value(problem.objective))

# for v in problem.variables():
#     print(v.name, "=", v.varValue)


Status: Optimal
Total Carbon Footprint =  1421.5248999999237
